<center><b><H1> KAGGLE COMPETITION GNR 638 </H1></b></center>
<center><b><H2> VINIT AWALE </H2></b></center>
<center><b><H2> 18D070067 </H2></b></center>
<center><b><H2> ELECTRICAL DUAL DEGREE </H2></b></center>

## Split the training folder to train and validation folders


In [4]:
import splitfolders

splitfolders.ratio('Dataset\\gnr6382021\\train', output='Dataset\\gnr6382021\\train', seed=1337, ratio=(.8, .2))  # 80% training, 20% validation

Copying files: 500 files [00:00, 584.89 files/s]


After this the folder structure is adjusted to have the following structure:


```
Dataset
├── train
│   ├──basketball_court
│   ├──bridge
│   ├──crosswalk   
│   ├──golf_course
│   ├──oil_well
│   ├──overpass
│   ├──railway
|   ├──runway
|   ├──swimming_pool
|   ├──tennis_court
|── val
│   ├──basketball_court
│   ├──bridge
│   ├──crosswalk
│   ├──golf_course
│   ├──oil_well
│   ├──overpass
│   ├──railway
|   ├──runway
|   ├──swimming_pool
|   ├──tennis_court
└── test


## Imports

In [2]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, utils
from torch.utils.data.sampler import SubsetRandomSampler
import pathlib


## Setting the device

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Transforms

In [4]:
transformer = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),    
    transforms.ToTensor(),                  # make it into torch.Tensor and normalized into range [0, 1]
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) # transform it into range [-1, 1]
])

## Dataloader

In [5]:
train_path = "Dataset\\train"
val_path = "Dataset\\val"


train_loader = DataLoader(torchvision.datasets.ImageFolder(train_path, transform=transformer), batch_size=32, shuffle=True)
val_loader = DataLoader(torchvision.datasets.ImageFolder(val_path, transform=transformer), batch_size=32, shuffle=True)

In [6]:
## Categories in the dataset
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [7]:
classes

['basketball_court',
 'bridge',
 'crosswalk',
 'golf_course',
 'oil_well',
 'overpass',
 'railway',
 'runway',
 'swimming_pool',
 'tennis_court']

## Building the CNN

In [8]:
VGG_types = {
    "VGG11": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG13": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG16": [64,64,"M",128,128,"M",256,256,256,"M",512,512,512,"M",512,512,512,"M"],
    "VGG19": [64,64,"M",128,128,"M",256,256,256,256,"M",512,512,512,512,"M",512,512,512,512,"M"],
}


class VGG_net(nn.Module):
    def __init__(self, in_channels, num_classes, type="VGG16"):
        super(VGG_net, self).__init__()
        self.in_channels = in_channels
        self.conv_layers = self.create_conv_layers(VGG_types[type])

        self.fcs = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fcs(x)
        return x

    def create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels

        for x in architecture:
            if type(x) == int:
                out_channels = x

                layers += [
                    nn.Conv2d(
                        in_channels=in_channels,
                        out_channels=out_channels,
                        kernel_size=(3, 3),
                        stride=(1, 1),
                        padding=(1, 1),
                    ),
                    nn.BatchNorm2d(x),
                    nn.ReLU(),
                ]
                in_channels = x
            elif x == "M":
                layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))]

        return nn.Sequential(*layers)

In [12]:
num_class = len(classes) # Number of classes in the dataset

model = VGG_net(3,num_class,"VGG16").to(device)
print(model)
## N = 3 (Mini batch size)
# x = torch.randn(3, 3, 224, 224).to(device)
# print(model(x).shape)

VGG_net(
  (conv_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3),

## Optimizer and Loss

In [13]:
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
loss_func = nn.CrossEntropyLoss()

In [14]:
num_epochs = 10

In [17]:
## Calculate number of training and validation images
num_train = len(glob.glob(train_path + "/**/*.jpg"))
num_val = len(glob.glob(val_path + "/**/*.jpg"))

In [18]:
print("Number of training images: {}".format(num_train))
print("Number of validation images: {}".format(num_val))

Number of training images: 500
Number of validation images: 100


## Model training and savint the best model